- definir os caracteres da linguagem e atribuir um fonema pra cada um (e categorizar vogais, consoantes e consoantes conectivas).
- definir regras de conexões entre os caracteres.
- gerar uma lista com todas as possíveis sílabas da linguagem a partir dos caracteres e das regras definidas.
- escolher um corpus para a língua e aplicar um histograma de frequência de palavras.
- fazer tratamento do corpus com stemização (radicalização) e análise de prefixos, sufixos e temporalidade.
- gerar radicais e sufixos na língua nova combinando sílabas aleatórias (atribuir combinações com menos sílabas às palavras mais frequentes do corpus).

In [1]:
import re, os, sys
import numpy as np
import pandas as pd
import itertools
import random
import pickle
from tqdm import tqdm

from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import wordpunct_tokenize
from datasets import load_dataset

random.seed(1)

In [2]:
vogais = [
    1,      # Atum, pAta, mArca, cAbra, mArte.
    2,      # metA, lendA, ecA, lidA, jantA, clavA.
    3,      # jAnta, órfÃ, Anta, tAnta, pÃo.
    4,      # Ermo, lEr, vErde, crÊ, caratÊ.
    5,      # fEno, venEno, grafEno.
    6,      # É, chulÉ, pErto, mErda, tEste, crEdo.
    7,      # Idade, perdI, mIngau, pIada.
    8,      # hOrta, pOrta, Ódio.
    9,      # Ortopédico, hOspital, Oligarquia, pivÔ.
    10,     # Onda, pÕe, fogÕes.
    11,     # bUsca, Unidade, Ungido, angU.
]

consoantes = [
    12,     # Brotar, Binóculo, Busca.
    13,     # Casa, Credo, aCatar, paCato.
    14,     # Cedo, Seda, peSte, pSicólogo, caÇar, haSte.
    15,     # Dedo, Domar, oDontológico, Dinossauro.
    16,     # Fada, Feio, inFinito.
    17,     # (TH do inglês) THere, THis, wiTH, THeory.
    18,     # Grande, aGradar, GUiar.
    19,     # aGiota, aGir, viaJar, Jeito, Girafa.
    20,     # Linda, Lama, Luar, aLisar.
    22,     # LHama, aLHo, veLHo, boLHa.
    23,     # Mingau, Miado, aMar, aMigo, cagaM.
    24,     # Ninho, Nada, aNta, póleN, teNda.
    25,     # niNHo, NHaca, diNHeiro, neNHum.
    26,     # Pata, Perto, aPanhar.
    27,     # araR, Rato, RoeR, aRte, aRRanhaR.
    28,     # (Mesmo R acima, na garganta, mas estendido RRR, usar caracteres parecidos).
    29,     # aRar, faRá, aRaRa, caiçaRa.
    30,     # (Mesmo R acima, na língua, mas estendido RRR, usar caracteres parecidos).
    31,     # Terno, TesTe, Treme, aTuar, aTum.
    32,     # Vida, Verde, aVoar.
    33,     # CHá, CHato, Xícara, taXa.
    34,     # eXame, deSânimo, meSa, Zebra, Zangão, eXagero.
    35,     # (V do espanhol, parecido com B)
]

In [3]:
class Linguagem():
    def __init__(self, vogais, consoantes, especiais, regras_silabais, precedencias_silabas, df_palavras_ptbr=None):
        self.vogais = vogais
        self.consoantes = consoantes
        self.especiais = especiais
        self.regras_silabais = regras_silabais
        self.precedencias_silabas = precedencias_silabas

        self.dict_tipo_char = {"v": self.vogais, "c": self.consoantes, "e": self.especiais}

        self.silabas_dict, self.silabas_list = self.gerar_silabas()
        self.dicionario_pt_lin = dict()
        self.dicionario_lin_pt = dict()
        self.dicionario_radical_pt_lin = dict()
        self.dicionario_radical_lin_pt = dict()
        # self.df_palavras = None if df_palavras_ptbr is None else transform_df(df_palavras_ptbr)

    def gerar_silabas(self):
        total = 0
        silabas_dict = dict()
        silabas_list = list()
        for n, regra in self.regras_silabais.items():
            regra_caracteres = list()
            for tipo_char in regra:
                regra_caracteres.append(self.dict_tipo_char[tipo_char])
            silabas = list(itertools.product(*regra_caracteres))
            silabas_dict[n] = silabas
            silabas_list += silabas
        return silabas_dict, silabas_list

    def gerar_palavra_aleatoria(self, n_silabas):
        palavra = list()
        tipo_silaba = 0
        for idx in range(n_silabas):
            tipo_silaba = random.choice(self.precedencias_silabas[tipo_silaba])
            palavra.append(random.choice(self.silabas_dict[tipo_silaba]))
        return palavra

    # def transform_df(df_palavras):
    #     n_silabas_atual = 1
    #     for row in df_palavras_ptbr.iterrows():
    #         if row[1]["radical_lin"] == "":

In [4]:
lingua_simples_teste_1 = (
    Linguagem(
        [2, 3, 4, 7, 9],
        [13, 17, 19, 20, 23, 24, 25, 27, 35],
        [14, 16, 32],
        {1: "cv", 2: "cev", 3: "cve", 4: "cvv", 5: "ev", 6: "ve", 7: "v"},
        {0: [1, 2, 3, 4, 5, 6, 7], 1: [1, 2, 3, 4, 5], 2: [1, 2, 3, 4, 5], 3: [1, 2, 3, 4, 5], 4: [1, 2, 3, 4, 5], 5: [1, 2, 3, 4, 5], 6: [1, 2, 3, 4, 5], 7: [1, 2, 3, 4, 5]},
    )
)

575


{1: [(13, 2),
  (13, 3),
  (13, 4),
  (13, 7),
  (13, 9),
  (17, 2),
  (17, 3),
  (17, 4),
  (17, 7),
  (17, 9),
  (19, 2),
  (19, 3),
  (19, 4),
  (19, 7),
  (19, 9),
  (20, 2),
  (20, 3),
  (20, 4),
  (20, 7),
  (20, 9),
  (23, 2),
  (23, 3),
  (23, 4),
  (23, 7),
  (23, 9),
  (24, 2),
  (24, 3),
  (24, 4),
  (24, 7),
  (24, 9),
  (25, 2),
  (25, 3),
  (25, 4),
  (25, 7),
  (25, 9),
  (27, 2),
  (27, 3),
  (27, 4),
  (27, 7),
  (27, 9),
  (35, 2),
  (35, 3),
  (35, 4),
  (35, 7),
  (35, 9)],
 2: [(13, 14, 2),
  (13, 14, 3),
  (13, 14, 4),
  (13, 14, 7),
  (13, 14, 9),
  (13, 16, 2),
  (13, 16, 3),
  (13, 16, 4),
  (13, 16, 7),
  (13, 16, 9),
  (13, 32, 2),
  (13, 32, 3),
  (13, 32, 4),
  (13, 32, 7),
  (13, 32, 9),
  (17, 14, 2),
  (17, 14, 3),
  (17, 14, 4),
  (17, 14, 7),
  (17, 14, 9),
  (17, 16, 2),
  (17, 16, 3),
  (17, 16, 4),
  (17, 16, 7),
  (17, 16, 9),
  (17, 32, 2),
  (17, 32, 3),
  (17, 32, 4),
  (17, 32, 7),
  (17, 32, 9),
  (19, 14, 2),
  (19, 14, 3),
  (19, 14, 4),
  (

In [5]:
lingua_simples_teste_1.gerar_palavra_aleatoria(1)

[(17, 14, 3)]

thsã

In [6]:
lingua_simples_teste_1.gerar_palavra_aleatoria(2)

[(19, 2, 14), (27, 7, 9)]

jãsrriô

In [7]:
lingua_simples_teste_1.gerar_palavra_aleatoria(3)

[(23, 9, 2), (35, 2, 3), (17, 16, 9)]

môãvãáthfô

In [8]:
corpus_carolina = [
    load_dataset("carolina-c4ai/corpus-carolina", taxonomy="dat"), 
    load_dataset("carolina-c4ai/corpus-carolina", taxonomy="pub"), 
    load_dataset("carolina-c4ai/corpus-carolina", taxonomy="wik"), 
    load_dataset("carolina-c4ai/corpus-carolina", taxonomy="soc"), 
    load_dataset("carolina-c4ai/corpus-carolina", taxonomy="uni")
]

In [9]:
def term_frequency_from_corpus(corpus_list):
    for corpus in corpus_list:
        term_frequency = dict()
        for text in tqdm(corpus["corpus"]["text"]):
            for word in wordpunct_tokenize(text):
                if not word.isalpha():
                    continue
                word = word.lower()
                term_frequency[word] = term_frequency[word] + 1 if word in term_frequency.keys() else 1
    return sorted(term_frequency.items(), key=lambda x:x[1], reverse=True)

In [10]:
if "term_frequency.pkl" not in os.listdir("./checkpoints/"):
    term_frequency = term_frequency_from_corpus(corpus_carolina)
    with open("./checkpoints/term_frequency.pkl", "wb") as file:
        pickle.dump(term_frequency, file)
else:
    with open("./checkpoints/term_frequency.pkl", "rb") as file:
        term_frequency = pickle.load(file)

In [11]:
print(len(term_frequency))
term_frequency

55342


[('de', 56733),
 ('a', 36697),
 ('e', 35547),
 ('o', 27686),
 ('que', 26082),
 ('da', 18919),
 ('do', 17307),
 ('em', 16134),
 ('para', 11194),
 ('se', 10666),
 ('com', 10005),
 ('um', 9677),
 ('no', 9164),
 ('é', 9090),
 ('os', 8854),
 ('uma', 8666),
 ('não', 7824),
 ('por', 7513),
 ('as', 7474),
 ('na', 7462),
 ('como', 7258),
 ('dos', 5707),
 ('mais', 5217),
 ('ao', 4894),
 ('das', 4663),
 ('ou', 4195),
 ('à', 4059),
 ('são', 3688),
 ('sua', 3054),
 ('mas', 2984),
 ('ser', 2971),
 ('nos', 2934),
 ('foi', 2585),
 ('usp', 2548),
 ('pela', 2335),
 ('sobre', 2266),
 ('entre', 2214),
 ('seu', 2182),
 ('também', 2157),
 ('pelo', 2093),
 ('brasil', 1857),
 ('nas', 1705),
 ('ainda', 1695),
 ('já', 1653),
 ('anos', 1629),
 ('mesmo', 1593),
 ('seus', 1513),
 ('isso', 1401),
 ('aos', 1391),
 ('paulo', 1373),
 ('ele', 1360),
 ('há', 1360),
 ('muito', 1346),
 ('sem', 1342),
 ('quando', 1325),
 ('era', 1303),
 ('suas', 1302),
 ('até', 1299),
 ('mundo', 1273),
 ('pode', 1270),
 ('tem', 1227),
 ('à

In [12]:
df_palavras_ptbr = pd.DataFrame(term_frequency, columns=["palavra_pt", "frequencia"])

In [13]:
stemmer = SnowballStemmer(language='portuguese')
df_palavras_ptbr["radical_pt"] = df_palavras_ptbr["palavra_pt"].apply(stemmer.stem)
df_palavras_ptbr["radical_lin"] = ""
df_palavras_ptbr["palavra_lin"] = ""
df_palavras_ptbr.head(50)

,palavra_pt,frequencia,radical_pt,radical_lin,palavra_lin
0,de,56733,de,,
1,a,36697,a,,
2,e,35547,e,,
3,o,27686,o,,
4,que,26082,que,,
5,da,18919,da,,
6,do,17307,do,,
7,em,16134,em,,
8,para,11194,par,,
9,se,10666,se,,
